In [1]:
import re2 as re
import json
import os
import collections
import pickle
import unicodedata
import sys
# import re
import tqdm
from ufal.udpipe import Model, Pipeline

In [31]:
# data_dir_list = ['dataset/AA']
data_dir_list = ['dataset/AA','dataset/AB']
# data_dir_list = ['dataset/AA','dataset/AB','dataset/AC','dataset/AD','dataset/AE','dataset/AF']
tmp_dir = 'outputs/tmp'
results_dir = 'outputs/results' 
udpipe_model = 'udpipe/russian-ud-2.0-170801.udpipe' 
tokens_file = 'tokens_set.pkl'
chars_file = 'chars_set.pkl'

In [32]:
postfix = os.path.split(data_dir_list[-1])[-1]
tmp_dir = os.path.join(tmp_dir, postfix)
tokens_file = os.path.join(tmp_dir, tokens_file)
chars_file = os.path.join(tmp_dir, chars_file)
os.makedirs(tmp_dir, exist_ok = True)
os.makedirs(results_dir, exist_ok = True)

In [33]:
json_files = []
for data_dir in data_dir_list:
    json_files.extend([os.path.join(data_dir,file_path) for file_path in  os.listdir(data_dir)])

In [34]:
topics = []
for file in json_files:
    for line in open(file, 'r'):
        topics.append(json.loads(line))

In [35]:
texts = [topic['text'] for topic in topics]

In [9]:
def counters_merge(counters):
    if len(counters) < 3:
        share_counter = collections.Counter()
        for counter in counters:
            share_counter += counter
        return share_counter
    else:
        split_point = len(counters)//2
        l_c = counters_merge(counters[:split_point])
        r_c = counters_merge(counters[split_point:])
        return l_c + r_c
    
def count_tokens(text_list):
    counters = []
    for text in tqdm.tqdm(text_list):
        counters.append(collections.Counter(text))
    return counters_merge(counters)
def count_chars(text_list):
    text_list = [''.join(text) for text in text_list]
    return count_tokens(text_list)

In [71]:
def first_preproc(text_list, modelfile):
    model = Model.load(modelfile)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'horizontal')
    combining_characters = dict.fromkeys([c for c in range(sys.maxunicode)
                                                   if unicodedata.combining(chr(c))])
    sanitized_texts = []
    for text in tqdm.tqdm(text_list):
        ref_name_tag_regexp = r'<ref name=.*?>'
        div_class_tag_regexp = r'<div class=.*?>'
        other_tag_regexp = r'</?\s?[^(math)(/math)(…)\.А-Яа-я0-9\"].{0,150}?>'
        repeat_math_tag_regexp = r'(</?math>[^А-Яа-я]{0,150}?)</?math>'
        math_tag_regexp = r'</?math*?>'
        url_regexp = r'(?i)\b((?:(https?|ftp)://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb]))'
        text = re.sub(ref_name_tag_regexp, r" ", text) #remove <ref name=
        text = re.sub(div_class_tag_regexp, r" ", text) #remove <div class=
        text = re.sub(other_tag_regexp, r" ", text) #remove other tags
        text = '<neli>'.join(text.split('\n'))
        text = re.sub(repeat_math_tag_regexp, r" ", text) # degree of nesting 3 math tags
        text = re.sub(repeat_math_tag_regexp, r" ", text)
        text = re.sub(repeat_math_tag_regexp, r" ", text)
        text = '\n'.join(text.split('<neli>'))
        text = re.sub(math_tag_regexp, r" ", text) #remove all math tags
        #----------------
        text = re.sub(r'…', r'...', text) # … -> ... (2026)
        text = re.sub(r'\?{1,}![\?!]+', r'?!', text) # ?!???!! -> ?!
        text = re.sub(r'!{1,}\?[\?!]+', r'?!', text) # ?!???!! -> ?!
        text = re.sub(r'!{3,}', r'!!', text) # !!!!!!!! -> !!
        text = re.sub(r'\?{3,}', r'??', text) # ???? -> ??
        text = re.sub(r'\.{4,}', r'...', text) # ....... -> ...
        text = re.sub(r'[“”«»]', r'"', text) # 
        text = re.sub(r"’", r"'", text) #
        text = re.sub(r"[`']{2,}", r'"', text) # 
        text = re.sub(r'[‐‑‒–—―-]{1,}', r'-', text) # (2010)(2011)(2012)(2013)(2014)(2015)(2016) -> - (2012)
        #----------------  
        text = re.sub(url_regexp, r" bracketURLbracket ", text)
        text = re.sub(r"\s+", r" ", text)
        text = re.sub(r'­', r'', text)
        sentences = pipeline.process(text).split('\n')
        sanitized_sentences = []
        for sentence in sentences:
            tokens = sentence.split()
            tokens = [token if token != 'bracketURLbracket' else '<URL>' for token in tokens]
            sanitized_tokens = []
            for token in tokens:
                decomposed_token = unicodedata.normalize('NFD', token)
                sanitized_token = decomposed_token.translate(combining_characters)
                # Move bak reversed N with hat
                sanitized_token = ''.join(ch_san if ch not in 'йЙ' else ch for ch, ch_san in zip(token, sanitized_token))
                sanitized_token = unicodedata.normalize('NFC', sanitized_token)
                
                sanitized_tokens.append(sanitized_token)
            sanitized_sentences.append(sanitized_tokens)
        sanitized_texts.append(sanitized_sentences)
    return sanitized_texts

In [72]:
%time sanitized_texts = first_preproc(texts[:10],udpipe_model)


 70%|███████   | 7/10 [00:07<00:03,  1.05s/it]
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/denis/env/share/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/denis/env/share/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration
100%|██████████| 10/10 [00:13<00:00,  1.32s/it]

CPU times: user 14.1 s, sys: 40 ms, total: 14.2 s
Wall time: 14.2 s


In [76]:
sanitized_texts[0][0]

['Астана',
 'Астана',
 '(',
 '—',
 '«столица»',
 ';',
 'до',
 '1961',
 'года',
 '—',
 'Акмолинск',
 ',',
 'в',
 '1961—1992',
 'годах',
 '—',
 'Целиноград',
 ',',
 'в',
 '1992—1998',
 'годах',
 '—',
 'Акмола',
 ')',
 '—',
 'столица',
 'Республики',
 'Казахстан',
 'с',
 '10',
 'июня',
 '1998',
 'года',
 '.']

In [44]:
%time char_counter = count_chars(sanitized_texts)

100%|██████████| 200/200 [00:00<00:00, 1357.56it/s]

CPU times: user 178 ms, sys: 7.99 ms, total: 186 ms
Wall time: 183 ms


In [45]:
%time token_counter = count_tokens(sanitized_texts)

100%|██████████| 200/200 [00:00<00:00, 3555.66it/s]


CPU times: user 379 ms, sys: 5 µs, total: 379 ms
Wall time: 376 ms


In [84]:
token_counter['­']

0

In [79]:
char_counter['­']

29

In [80]:
char_counter.most_common()

[('о', 197518),
 ('е', 155312),
 ('и', 148942),
 ('а', 134368),
 ('н', 121113),
 ('т', 105920),
 ('с', 102166),
 ('р', 94661),
 ('в', 81609),
 ('л', 75927),
 ('к', 57578),
 ('м', 53626),
 ('д', 50824),
 ('п', 46128),
 ('у', 37707),
 ('я', 36869),
 ('ы', 33914),
 ('г', 32710),
 ('з', 28934),
 (',', 26123),
 ('й', 24963),
 ('б', 23959),
 ('ь', 23894),
 ('ч', 21536),
 ('.', 18606),
 ('х', 17345),
 ('ж', 14237),
 ('ю', 11418),
 ('ц', 10752),
 ('ш', 8750),
 ('1', 7465),
 ('ф', 6518),
 ('щ', 6501),
 ('В', 5796),
 ('0', 5411),
 ('э', 5215),
 ('С', 5091),
 ('П', 4671),
 (')', 4107),
 ('(', 4070),
 ('—', 3985),
 ('2', 3964),
 ('-', 3432),
 ('9', 3308),
 ('К', 3068),
 ('«', 3066),
 ('»', 3066),
 ('А', 2528),
 ('М', 2511),
 ('Н', 2323),
 ('5', 2274),
 ('О', 2268),
 ('8', 2172),
 ('4', 2118),
 ('3', 2096),
 ('a', 2067),
 ('Р', 2056),
 ('"', 2046),
 ('Г', 1988),
 ('6', 1951),
 ('Т', 1914),
 ('7', 1897),
 ('Б', 1842),
 ('e', 1820),
 ('r', 1781),
 ('o', 1750),
 ('Л', 1665),
 ('И', 1652),
 ('Д', 1602)

In [61]:
pickle.dump(char_counter, open(chars_file, 'wb'))

In [87]:
char_counter['ё']

0

In [15]:
def search_diversity(token, before_n = 0, after_n = 0):
    diversity_list = []
    for tx_i, text in enumerate(sanitized_texts):
        for tk_i, _token in enumerate(text):
            if token == _token:
                diversity_list.append(' '.join(sanitized_texts[tx_i][tk_i - before_n:tk_i + after_n]))
    return collections.Counter(diversity_list)

In [101]:
import regex as rgx
import re as rgx
def my_regexp(text_list):
    combining_characters = dict.fromkeys([c for c in range(sys.maxunicode)
                                                   if unicodedata.combining(chr(c))])
    sanitized_texts = []
    for text in text_list:
        regexp = rgx.compile(r'­')
        for token in text.split():
            if len(regexp.findall(token)) and regexp.findall(token)[0]:
                print(regexp.findall(token))
                print(token)
                print(re.sub(r'­','',token))


In [102]:
search_diversity('URL', before_n = 1, after_n=5).most_common()

[]

In [103]:
my_regexp(texts)

['\xad', '\xad']
гидро­пере­дачей
гидропередачей
['\xad']
паровозо­строитель
паровозостроитель
['\xad']
локомотиво­строительным
локомотивостроительным
['\xad']
локомотиво­строительных
локомотивостроительных
['\xad']
зна­чения
значения
['\xad']
тём­ный
тёмный
['\xad']
специ­альные
специальные
['\xad']
"Ес­ли
"Если
['\xad', '\xad', '\xad']
Ги­ор­гад­зе»
Гиоргадзе»
['\xad', '\xad', '\xad', '\xad']
спе­ци­аль­но­го
специального
['\xad', '\xad', '\xad']
на­зна­че­ния
назначения
['\xad']
пропаган­ду
пропаганду
['\xad']
Про­ект
Проект
['\xad']
си­стеме,
системе,
['\xad']
дво­ичную
двоичную
['\xad']
представ­лены
представлены
['\xad']
отверсти­ями
отверстиями
['\xad']
предполага­лось
предполагалось
['\xad']
различа­вшего
различавшего
['\xad']
диску­рсивное
дискурсивное
['\xad']
взгля­дом»
взглядом»
['\xad']
интернет-­газету
интернет-газету
['\xad']
«Вести-­Приморье»
«Вести-Приморье»
['\xad']
остро­в
остров
['\xad']
ко­торую
которую
['\xad']
учи­тельская
учительская
['\xad']
сан­сары»:
сансары»

In [25]:
def p_rgxp(reg):
    l = []
    tmpl = re.compile(reg)
    for text in texts:
        text = '<nl>'.join(text.split('\n'))
        r = tmpl.findall(text)
        if r:
            l.extend(r)
    return collections.Counter(l)

In [37]:
# pattern1 = 
# </ref>
# </math>
# <br>
# p_rgxp(r'</?[^(math)(/math)(\.\.\.)].*?>').most_common()
# p_rgxp(r'</?\s?[^(math)(/math)(…)\.А-Яа-я0-9\"].{0,100}?>').most_common()
# (p_rgxp(r'</?.*?>') - p_rgxp(r'</?\s?[^(math)(/math)(…)\.А-Яа-я0-9\"].{0,100}?>') - p_rgxp(r'<div class=.*?>') - p_rgxp(r'<ref name=.*?>')).most_common()
# p_rgxp(r'<ref name=.*?>').most_common()
# p_rgxp(r'<div class=.*?>').most_common()
# (p_rgxp(r'</?\s?[^(math)(/math)(…)\.А-Яа-я0-9\"].{0,100}?>') + p_rgxp(r'<div class=.*?>') + p_rgxp(r'<ref name=.*?>')).most_common()

# (p_rgxp(r'/') - p_rgxp(r'</?\s?[^(math)(/math)(…)\.А-Яа-я0-9\"].{0,100}?>') - p_rgxp(r'<div class=.*?>') - p_rgxp(r'<ref name=.*?>')).most_common()

# p_rgxp(r'<div class=.*?>').most_common()
# p_rgxp(r'/[A-Z-a-z\W]{1,}')
p_rgxp(r'(</?math>[^А-Яа-я]{0,150}?)</?math>').most_common()
# p_rgxp(r'</?math>').most_common()
# len(p_rgxp(r'[A-Za-z\W0-9_]{0,20}\\frac[\W]+[A-Za-z\W0-9_]*').most_common())

[('</math><nl>5. formula_35<nl>\\frac{np^{n+2}-(n+1)p^{n+1}+p}{(1-p)^2}', 1)]

In [24]:
texts[0]

'Астана\n\nАстана́ ( — «столица»; до 1961 года — Акмо́линск, в 1961—1992 годах — Целиногра́д, в 1992—1998 годах — Акмола́) — столица Республики Казахстан с 10 июня 1998 года. Акмолинск получил статус города 26 сентября 1862 года. Население Астаны по состоянию на 1 мая 2016 года составляло 880 191 человек, что являлось третьим показателем в Казахстане после Алма-Аты и Шымкента. 4 июля 2016 года было объявлено о том, что в Астане родился миллионный житель. Официальное статистическое агентство Казахстана Казстат не подтвердило данную дату достижения численности населения, несмотря на то, что КазСтат, по заявлению его руководителей, изменил методику оценки численности населения, что привело к статистическому увеличению оценки численности населения. Даже через полгода население Астаны составляло 972 672 человека (на 1 января 2017 года) и превысило 1 миллион лишь в июне 2017 года, когда составило 1 002 874 жителей. На 1 февраля 2018 года население города (исчисленное по новой методике) соста

In [53]:
import nltk

In [83]:
nltk.word_tokenize('­')

['\xad']

In [58]:
len(texts)

16014

In [78]:
open('1','wt').write('asd\nqwe')

7

In [87]:
re.search(r't', '123123123tte123123t12312312xt')

<re2.Match object; span=(9, 10), match='t'>